In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd

# DATASETS SLPLITING

In [3]:
import numpy as np
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

def load_descriptions(doc):
    mapping = dict()
    # process lines
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        if len(line) < 2:
            continue
        # take the first token as the image id, the rest as the description
        image_id, image_desc = tokens[0], tokens[1:]
        # extract filename from image id
        image_id = image_id.split('.')[0]
        # convert description tokens back to string
        image_desc = ' '.join(image_desc)
        # create the list if needed
        if image_id not in mapping:
            mapping[image_id] = list()
        # store description
        mapping[image_id].append(image_desc)
    return mapping

# load a pre-defined list of photo identifiers
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    # process line by line
    for line in doc.split('\n'):
        # skip empty lines
        if len(line) < 1:
            continue
        # get the image identifier
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)



# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
    
    # load document
    doc = load_doc(filename)
    descriptions = dict()
    
    i=0
    
    descriptions["id"] = list()
    descriptions["label"] = list()
    descriptions["image_id"] = list()
    descriptions["desc"] = list()
    
    # As the dataset is not prepared for a multimodal binary classification
    # let's mix some instances and make the label '0' means that the image 
    # and the text are similar and the laber '1' means that are not similar
    lines = doc.split('\n')
    total_instances = len(dataset)
    positivos = int(0.8 * total_instances)
    negativos = total_instances - positivos

    for j, line in enumerate(lines):        
        # split line by white space
        tokens = line.split()
        
        # split id from description
        image_id, image_desc = tokens[0], tokens[1:]
        
        desc = ""
        for w in image_desc:
            desc = desc + " " + w
            
        if image_id in dataset:
            
            # create list
            descriptions["id"].append(i)
            descriptions["desc"].append(desc)
            
            if i<positivos*5:
                descriptions["image_id"].append(image_id)
                descriptions["label"].append(0)

            else:
                original_state = j
                j = j-10
                tokens = lines[j].split()
                
                # split id from description
                image_id, image_desc = tokens[0], tokens[1:]
                
                while image_id not in dataset:
                    j = j-5
                    tokens = lines[j].split()
                    # split id from description
                    image_id, image_desc = tokens[0], tokens[1:]
                
                descriptions["image_id"].append(image_id)
                descriptions["label"].append(1)  
                    
                j = original_state
                
            i+=1

    return descriptions

In [4]:
doc = load_doc("./descriptions.txt")
doc = load_descriptions(doc)

train_set = load_set("E://TFM/Flickr8k/Flickr8k_text/Flickr_8k.trainImages.txt")
train_set = load_clean_descriptions("./descriptions.txt",train_set)

test_set = load_set("E://TFM/Flickr8k/Flickr8k_text/Flickr_8k.testImages.txt")
test_set = load_clean_descriptions("./descriptions.txt",test_set)

val_set = load_set("E://TFM/Flickr8k/Flickr8k_text/Flickr_8k.devImages.txt")
val_set = load_clean_descriptions("./descriptions.txt",val_set)

df = pd.DataFrame(data=doc)
df_train = pd.DataFrame(data=train_set)
df_test = pd.DataFrame(data=test_set)
df_val = pd.DataFrame(data=val_set)

In [9]:
df_train.head()

,id,label,image_id,desc
0,0,0,1000268201_693b08cb0e,child in pink dress is climbing up set of sta...
1,1,0,1000268201_693b08cb0e,girl going into wooden building
2,2,0,1000268201_693b08cb0e,little girl climbing into wooden playhouse
3,3,0,1000268201_693b08cb0e,little girl climbing the stairs to her playhouse
4,4,0,1000268201_693b08cb0e,little girl in pink dress going into wooden c...


In [6]:
df_test.head()

,id,label,image_id,desc
0,0,0,1056338697_4f7d7ce270,blond woman in blue shirt appears to wait for...
1,1,0,1056338697_4f7d7ce270,blond woman is on the street hailing taxi
2,2,0,1056338697_4f7d7ce270,woman is signaling is to traffic as seen from...
3,3,0,1056338697_4f7d7ce270,woman with blonde hair wearing blue tube top ...
4,4,0,1056338697_4f7d7ce270,woman in the blue dress is holding out her ar...


In [7]:
df_val.head()

,id,label,image_id,desc
0,0,0,1022454332_6af2c1449a,child and woman are at waters edge in big city
1,1,0,1022454332_6af2c1449a,large lake with lone duck swimming in it with...
2,2,0,1022454332_6af2c1449a,little boy at lake watching duck
3,3,0,1022454332_6af2c1449a,young boy waves his hand at the duck in the w...
4,4,0,1022454332_6af2c1449a,people are at the edge of lake facing the wat...


# TENSORFLOW HUB

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)

In [10]:
def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
list_train = [df_train[i:i+100] for i in range(0,df_train.shape[0],100)]
list_test = [df_test[i:i+100] for i in range(0,df_test.shape[0],100)]

# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['desc']) for x in list_train]
elmo_test = [elmo_vectors(x['desc']) for x in list_test]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [ ]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()